# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('../WeatherPy/City_Data.csv');
cities_df.head(5)

,City_Name,Longitude,Lattitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Katsuura,140.3000,35.1333,74.25,86,100,9.51
1,Srostki,85.7022,52.4189,46.85,90,54,3.71
2,‘Āhuimanu,-157.8378,21.4447,83.05,57,20,8.05
3,São João da Barra,-41.0511,-21.6403,71.08,74,10,12.12
4,Leningradskiy,178.4167,69.3833,51.84,48,96,2.04


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configuring gmaps
gmaps.configure(api_key=g_key);

Locations = cities_df[['Lattitude' , 'Longitude']];
weights = cities_df['Humidity'];

#Generating the gmaps figure
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(Locations , weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

78.2186

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
#subsetting dataframe on the basis of temperature
less_80 = cities_df[cities_df['Temperature'] < 80 ];
great_70 = less_80[less_80['Temperature'] > 70];

#Filtering out cities with wind speed greater than 10 mph
pass_wind = great_70[great_70['Wind Speed'] < 10];

#Finding cities with zero cloudiness
hotel_df = pass_wind[pass_wind['Cloudiness'] == 0];

#determining the number of cities that fit our criteria
print(len(hotel_df));

10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [97]:
#renaming 'City_Name' column to 'City'
hotel_columns = [col for col in hotel_df.columns]
hotel_columns[0] = 'City'
hotel_df.columns = hotel_columns;

In [76]:
#Formatting lat and long values from hotel_df to fit the api call model
latitudes = [str(lat) for lat in hotel_df['Lattitude']];
longitudes = [str(lon) for lon in hotel_df['Longitude']];
geo_coordinates = [latitudes[i] +','+ longitudes[i] for i in range(0 , len(latitudes))];

['-23.4436,-51.8739',
 '43.4,5.05',
 '31.6315,-8.0083',
 '29.5833,-9.5',
 '-9.6658,-35.7353',
 '-12.974,40.5178',
 '40.2747,18.3368',
 '22.6906,58.5334',
 '36.9559,36.2',
 '31.61,34.7642']

In [78]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

Request = []; #stores api requests

#looping through geo_cooordinates to make succesive calls to google places
for coord in geo_coordinates:

    #Searching for nearest hotel 
    target_coordinates = coord
    target_search = 'Hotel'
    target_radius = 5000
    target_type = 'lodging'

    #parameter dictionary
    params = {
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type':target_type,
        'key': g_key
            }
    Request.append(requests.get(base_url , params=params));
    

In [106]:
#Converting Requests into Json
search_result = [req.json() for req in Request];

#Storing Hotel names in a list to be added to hotel_df
Hotel_Names = [];
for result in search_result:

    try:
        Hotel_Names.append(result['results'][0]['name'])
    except:
        Hotel_Names.append('No Result');



In [110]:
hotel_df['Hotel_Name'] = Hotel_Names;
hotel_df

<ipython-input-110-8c17a3250f8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel_Name'] = Hotel_Names;


,City,Longitude,Lattitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name,Hotel_Name
110,Sarandi,-51.8739,-23.4436,72.95,64,0,6.91,,Hotel Caiuá Express Maringá
138,Martigues,5.0500,43.4000,70.50,68,0,3.44,,Inter Hotel Clair Hotel
175,Marrakesh,-8.0083,31.6315,78.87,50,0,4.61,,Sofitel Marrakech Lounge & Spa
202,Tiznit Province,-9.5000,29.5833,70.83,49,0,1.14,,No Result
245,Maceió,-35.7353,-9.6658,74.64,78,0,3.44,,Jatiúca Hotel & Resort
266,Pemba,40.5178,-12.9740,75.18,70,0,9.95,,Avani Pemba Beach Hotel
284,Melendugno,18.3368,40.2747,77.38,78,0,6.91,,La Casina di Borgagne
303,Ibrā’,58.5334,22.6906,79.09,59,0,3.02,,Ibra Hotel
324,Erzin,36.2000,36.9559,75.02,80,0,0.72,,Hattusa Vacation Thermal Club Erzin
426,Kiryat Gat,34.7642,31.6100,74.88,74,0,2.01,,No Result


In [114]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name      #Contradictory instruction
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lng", "Ltt"]]

KeyError: 'Country'

In [118]:
# Add marker layer ontop of heat map
Hotel_Locations = hotel_df[['Lattitude' , 'Longitude']];
fig.add_layer(gmaps.marker_layer(locations=Hotel_Locations))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))